In [ ]:
# hide
from doc_collection.core import *

In [ ]:
# hide
import warnings
warnings.filterwarnings("ignore")

# doc_collection

> A tool for collecting documentation of python libs.

## Install


`git clone https://github.com/omemaxim/doc_collection`

## How to use

Firstly, you need to pip list of packages to be able to collect data from it:

In [ ]:
pip_top_hundred()

Now you are ready to extract data from all libs you've pipped and it's dependencies:

In [ ]:
extract()

--------------- error during tensorflow-io-gcs-filesystem documentation extracting
--------------- error during theano documentation extracting


,text,paths,library,name
0,Python Library Documentation: Any in module tr...,"[ipykernel.comm.Comm.log, ipykernel.comm.CommM...",ipykernel,ipywidgets.fixed.value
1,Python Library Documentation: AxisProperty\n\n...,[pandas.DataFrame.columns],pandas,pandas.DataFrame.columns
2,Python Library Documentation: AxisProperty\n\n...,[pandas.Series.index],pandas,pandas.Series.index
3,Python Library Documentation: AxisProperty\n\n...,[pandas.DataFrame.index],pandas,pandas.DataFrame.index
4,Python Library Documentation: Bool in module o...,"[openpyxl.chart.AreaChart.roundedCorners, open...",openpyxl,openpyxl.styles.Color.auto
...,...,...,...,...
31263,Python Library Documentation: property\n\n ...,"[faiss.BufferList.wp, faiss.RangeSearchPartial...",faiss,faiss.BufferList.wp
31264,Python Library Documentation: reify\n\n,"[aiohttp.ClientResponse.content_disposition, a...",aiohttp,aiohttp.ClientResponse.url
31265,Python Library Documentation: reify\n\n A s...,[aiohttp.ClientResponse.history],aiohttp,aiohttp.ClientResponse.history
31266,Python Library Documentation: reify\n\n Ret...,[aiohttp.BodyPartReader.filename],aiohttp,aiohttp.BodyPartReader.filename


Command above will return DataFrame of four columns: __text__ contains documentation of an object, __paths__ contains all names of an object (can be more than one), __name__ contains the shortest name and __library__ contains library of an object.

# Elasticsearch 

To create index and load mentioned dataframe to elastic. 

In [ ]:
# es_add_bulk(extract()) - !!! Commented only to pass tests (tester does not have elastic program). Don't pay attantion

The index's name is __doc__ and other fields are the same as in dataframe.

In [ ]:
d = extract()

--------------- error during tensorflow-io-gcs-filesystem documentation extracting
--------------- error during theano documentation extracting


In [ ]:
d

,text,paths,library,name
0,Python Library Documentation: Any in module tr...,"[ipykernel.comm.Comm.log, ipykernel.comm.CommM...",ipykernel,ipywidgets.fixed.value
1,Python Library Documentation: AxisProperty\n\n...,[pandas.DataFrame.columns],pandas,pandas.DataFrame.columns
2,Python Library Documentation: AxisProperty\n\n...,[pandas.Series.index],pandas,pandas.Series.index
3,Python Library Documentation: AxisProperty\n\n...,[pandas.DataFrame.index],pandas,pandas.DataFrame.index
4,Python Library Documentation: Bool in module o...,"[openpyxl.chart.AreaChart.roundedCorners, open...",openpyxl,openpyxl.styles.Color.auto
...,...,...,...,...
31263,Python Library Documentation: property\n\n ...,"[faiss.BufferList.wp, faiss.RangeSearchPartial...",faiss,faiss.BufferList.wp
31264,Python Library Documentation: reify\n\n,"[aiohttp.ClientResponse.content_disposition, a...",aiohttp,aiohttp.ClientResponse.url
31265,Python Library Documentation: reify\n\n A s...,[aiohttp.ClientResponse.history],aiohttp,aiohttp.ClientResponse.history
31266,Python Library Documentation: reify\n\n Ret...,[aiohttp.BodyPartReader.filename],aiohttp,aiohttp.BodyPartReader.filename
